# Robot Sensors

Kevin J. Walchko, 24 Sept 2017

---

Now let's talk about the iRobot Create 2 we are going to use. There are a variety of sensors available and we will use some, but not all of them.

## References

- [Dead Reckoning Wikipedia](https://en.wikipedia.org/wiki/Dead_reckoning)
- [Interrupts](http://raspi.tv/2013/how-to-use-interrupts-with-python-on-the-raspberry-pi-and-rpi-gpio-part-3)
- [wikipedia: Infrared](https://en.wikipedia.org/wiki/Infrared)
- [Wikipedia IMU](https://en.wikipedia.org/wiki/Inertial_measurement_unit)
- [Kalman Filter](https://en.wikipedia.org/wiki/Kalman_filter)

## Software

- [pycreate2 python driver you will use](https://pypi.python.org/pypi/pycreate2)
- [IMU python driver](https://pypi.python.org/pypi/nxp-imu)

## Setup

In [1]:
%matplotlib inline

from __future__ import division, print_function
import matplotlib.pyplot as plt
from math import pi
from IPython.display import HTML, display

# iRobot Create Sensors

<img src="pics/create.png" width="300px">

For this class we are using an iRobot Create 2 as our robot base. This robot is equiped with several sensors, but the ones we will use the most are:

- IR Sensors
    - Cliff detectors that detect the floor or the lack of one in the case of stairs
    - Light bump detectors (really they are proximity sensors) that can detect objects in front of the roomba out to ~8"
- Physical bump detectors which are switches that detect physical impact on the front bumper
- Encoders which return the amount of wheel rotation for the left and right wheels
- Internal sensors for voltage and current on the battery and various motors (e.g., wheels, vacuum brush, etc)
    - NiMH battery pack with 19 Ahr capacity. You can actually read this and determine how drained the Roomba is.

## Infrared Proximity

Infrared sensor are a very common type of proximity sensor. They basically send out a beam of modulated infrared light and looks for the returned signal. Sharp makes a wide range of IR sensors for various commercial applications with detection distances ranging from 2 cm to 6 m.

<img src="pics/ir_sensors.jpg" width="300px">

These sensor return a voltage that corresponds to a measured distance. The typical
sensor curve is shown below.

<img src="pics/ir_range_curve.png" width="400px">

- IR Issues
   - IR sensors are suseptable to being washed out by sunlight
   - Multipath: IR light from one sensor could be seen by another if the sensors are misaligned or the light bounces off a reflective surface
   - IR works best against surfaces that are orthogonal to the sensor, bright surfaces, and reflect IR. *Note:* cardboard is IR transparent, therefore you will not get a reflection off standard brown cardboard box material.

## Quaditure Encoders

Unfortunately, the Create doesn't have really good encoders like what 
is discussed here, but this will give you an idea of how they work.

<img src="pics/usdigital_encoder.jpg" width="300px">

There are many types of encoders, above is a US Digital encoder designed to
be mounted on a motor shaft.

<img src="pics/quadrature_encoder.gif" width="600px">

The optical encoder, shines a series of lights through an encoder disk and
the light is detected or not detected on the other side of the encoder disk
by some photoreceptors. A quadrature encoder has 2 signals, A and B, which
are phased such that they are *never* high or low at the same time. Depending
on the phase of the signals, the direction can be determined.

<img src="pics/quadrature_animation.gif" width="300px">

The animation shows the signals produced from the movement of the motor
shaft, with the encoder disk attached to it.

<img src="pics/quadrature_waveform.gif" width="300px">

Again, the wave form from A and B tells us if the wheel (motor shaft
and disk) are moving in the forward or reverse direction. Note that
forward/reverse are arbitrary and the engineer needs to determine
if CW or CCW is forward or reverse depending on how the sensor was
mounted to the robot.

<img src="pics/quadrature_resolution.gif" width="300px">

The resolution of the encoder is determine by how the 2 signals are
read.

- reading A and B on rising edge of A gives you the resolution of how many
  stripes there are on the disk
- reading on the rising and falling edge of A gives you twice the resolution
  of the number of stripes on the disk
- reading both A and B for both rising and falling edges gives you 4 times
  the resolution as the number of stripes on the disk

Now, obviously, the last option gives you the greatest resolution and the
best performance ... so why wouldn't you do it? If the speed of your
microcontroller is too slow and/or the speed of your wheel is too fast, you
could get stuck answering interrupts all the time and never doing anything
else. You have to balance your system constraints properly.

### Python Pseudo Code

```python

import time
from serial import Serial

count = 0
COUNTS_TO_METERS = 0.001  # this depends on the encoder system

def main_loop():
ser = Serial('/dev/tty.usbserial0', 115200)

while True:
  time.sleep(1)  # time depends on speed of robot
  position += count * COUNTS_TO_METERS
  count = 0

  # a super simple serial response to report position
  if ser.read() == 'p':
    ser.write(position)

# an interrupt that gets called every time A or B changes
# you can do this with RPi.GPIO on the raspberry pi
def interrupt_AB():
    A, B = readEncoderPins()
    if A ^ B == 1:# Odometry (move to sensors lesson)
      count += 1
    else:
      count -= 1
```

# Odometry (Dead Reckoning)

A variety of robots are able to determine the distance ($d$) travelled because they have encoders on their wheels (our roomba does) to track the angle the wheel has moved ($\phi$). This is similar to how cars all come with odometers to show the miles travelled. 

$$
d = r \phi
$$

Now the equation above should look familar. If the wheel turns all the way around ($2\pi$ radians), then that equation becomes the circumference of a circle. If the world was perfect (spoiler, it isn't), then you would get this:

![](pics/ideal-odometry.png)

By tracking how far each wheel moves, we should be able to determine if a robot is going straight forward, turning to the left in reverse, or whatever. Unfortunately, wheels slip, expecially when it turns, and the distance travelled becomes corrupted. This leads to errors in the true angle of the turn and leads to the following:

![](pics/real-odometry.png)

This, odometry by itself is often not useful. Even worse, tracking a robot's position by how its wheels perform is not reliable at best. This is one reason why GPS is so useful with robots.

![](pics/rover.jpg)

The Mars rover uses video odometry to calculate its pose. Stereo cameras identify common features to track (like we did earlier in the vision block) in both the left and right camera. Then computed each feature's 3d position and tracked it from frame to frame to determine the distance and velocity travelled. With proper camera systems, lots of calibration and math, these types of systems are very good.

## Encoder Issues

    - Wheel slip can cause the encoder to read more distance travelled than what the robot has actually travelled. 
    - Wheel slip is usually most problematic when a robot turns. The greater the turn, the more slip that occurs, the more the encoders will be off.
    - Obviously, robots that must move accross loose dirt (sand dunes on Mars), snow, or whatever will have a discrepency between wheel movement and distance travelled ... even when going straight. 

# Additional Sensors: IMU

<img src="pics/imu-iso.jpg" width="300px">

Our Creates also have an inertial measurement unit (IMU) attached to the i2c bus of the Raspberry Pi. It uses an interface written for this class called [nxp_imu](https://pypi.python.org/pypi/nxp-imu). Our IMU is actually composed of 2 different IC chips that do different things.

### FXOS8700 3-Axis Accelerometer/Magnetometer

- 2-3.6V Supply
- ±2 g/±4 g/±8 g adjustable acceleration range
- ±1200 µT magnetic sensor range
- Output data rates (ODR) from 1.563 Hz to 800 Hz
- 14-bit ADC resolution for acceleration measurements
- 16-bit ADC resolution for magnetic measurements

### FXAS21002 3-Axis Gyroscope

- 2-3.6V Supply
- ±250/500/1000/2000°/s configurable range
- Output Data Rates (ODR) from 12.5 to 800 Hz
- 16-bit digital output resolution
- 192 bytes FIFO buffer (32 X/Y/Z samples)

Simple example `python` code to read the IMU is:

```python
from __future__ import division, print_function
from nxp_imu import IMU

# set your accels to 4 g's range and gyros to 2000 dps
imu = IMU(gs=4, dps=2000)

# now grab some data
# each of these is an array of [x,y,z]
accel, mag, gyro = imu.get()
```

# Reading Create 2 Sensors

To work the iRobot Create 2, we will use a library called [pycreate2](https://pypi.python.org/pypi/pycreate2) which was developed for this class. I suggest you take a look at the examples to help you write your code.

```python
from pycreate2 import Create2

# let's create a roomba instance and let it know what serial port to use
bot = Create2(port='/dev/ttyS0')
bot.start()

# the roomba has several modes, we will stay in safe mode
bot.safe()

# let's read some sensors
sensors = bot.get_sensors()
print(sensors)
```

In the above code, `sensors` is a python `namedtuple` with the following keys and values:

| Sensor Key Name              | Range             | Index |
|------------------------------|-------------------|-------|
| bumps\_wheeldrops            | \[0-15\]          | 0     |
| wall                         | \[0-1\]           | 1     |
| cliff\_left                  | \[0-1\]           | 2     |
| cliff\_front\_left           | \[0-1\]           | 3     |
| cliff\_front\_right          | \[0-1\]           | 4     |
| cliff\_right                 | \[0-1\]           | 5     |
| virtual\_wall                | \[0-1\]           | 6     |
| overcurrents                 | \[0-29\]          | 7     |
| dirt\_detect                 | \[0-255\]         | 8     |
| ir\_opcode                   | \[0-255\]         | 9     |
| buttons                      | \[0-255\]         | 10    |
| distance                     | \[-322768-32767\] | 11    |
| angle                        | \[-322768-32767\] | 12    |
| charger\_state               | \[0-6\]           | 13    |
| voltage                      | \[0-65535\]       | 14    |
| current                      | \[-322768-32767\] | 15    |
| temperature                  | \[-128-127\]      | 16    |
| battery\_charge              | \[0-65535\]       | 17    |
| battery\_capacity            | \[0-65535\]       | 18    |
| wall\_signal                 | \[0-1023\]        | 19    |
| cliff\_left\_signal          | \[0-4095\]        | 20    |
| cliff\_front\_left\_signal   | \[0-4095\]        | 21    |
| cliff\_front\_right\_signal  | \[0-4095\]        | 22    |
| cliff\_right\_signal         | \[0-4095\]        | 23    |
| charger\_available           | \[0-3\]           | 24    |
| open\_interface\_mode        | \[0-3\]           | 25    |
| song\_number                 | \[0-4\]           | 26    |
| song\_playing                | \[0-1\]           | 27    |
| oi\_stream\_num\_packets     | \[0-108\]         | 28    |
| velocity                     | \[-500-500\]      | 29    |
| radius                       | \[-322768-32767\] | 30    |
| velocity\_right              | \[-500-500\]      | 31    |
| velocity\_left               | \[-500-500\]      | 32    |
| encoder\_counts\_left        | \[-322768-32767\] | 33    |
| encoder\_counts\_right       | \[-322768-32767\] | 34    |
| light\_bumper                | \[0-127\]         | 35    |
| light\_bumper\_left          | \[0-4095\]        | 36    |
| light\_bumper\_front\_left   | \[0-4095\]        | 37    |
| light\_bumper\_center\_left  | \[0-4095\]        | 38    |
| light\_bumper\_center\_right | \[0-4095\]        | 39    |
| light\_bumper\_front\_right  | \[0-4095\]        | 40    |
| light\_bumper\_right         | \[0-4095\]        | 41    |
| ir\_opcode\_left             | \[0-255\]         | 42    |
| ir\_opcode\_right            | \[0-255\]         | 43    |
| left\_motor\_current         | \[-322768-32767\] | 44    |
| right\_motor\_current        | \[-322768-32767\] | 45    |
| main\_brush\_current         | \[-322768-32767\] | 46    |
| side\_brush\_current         | \[-322768-32767\] | 47    |
| statis                       | \[0-3\]           | 48    |

So you can access the sensor values in one of 2 ways:

```python
d = sensors.distance
d = sensors[11]
```

## Bag Files

Typically when you build things (i.e., robots, airplanes, satellites, etc) you rarely get to test the entire thing. Instead, if you are building a targeting system for an aircraft, you work with pre-recorded data (from a radar that was flying in another aircraft) and ensure your targeting system interacts with that "canned" data properly.

We will do the same here. You will play with some pre-recorded data from the roomba and try to understand what is going on. This will help you when you get to work with the real roomba. `bagit` stores data in a json file with optional compression. The idea of `bagit` comes from [ROS bag files](http://wiki.ros.org/Bags). Basically you do:

- Create a bag file to save data
    - collect data from your robot and store it in a python `dict`
    - then use `BagWriter` object to write the `dict` to a file
    - if any of the data is a camera image, you have to tell `BagWriter` so It can properly handle the binary image data
- Read data from a bag file
    - use a `BagReader` to read a bag file
    - the `BagReader` will return a `dict` containing all of the data. *Note:* camera images are automagically returned to standard OpenCV `numpy` arrays.
    
```python
```

In [16]:
from the_collector.bagit import BagWriter
import time

filename = 'my_file.json'

# create the writer
bag = BagWriter()

# filename is the bag file we will save it too
# 'sam' and 'bob' are just the keys in the dict we will say data too
bag.open(['sam', 'bob'])

# let's make a bunch of bogus data ... pretend this is real robot stuff!
# when you push data to the bag, it records the data with a timestamp
for i in range(10):
    bag.push('bob', i)        # save some data
    bag.push('sam', [i, i+1]) # save some more data
    time.sleep(0.1)

bag.write(filename)  # now save the data

In [17]:
from the_collector.bagit import BagReader

reader = BagReader()
data = reader.load(filename)  # read in the file and conver to dict

# now print everything out
for key, value in data.items():
        print('-- {} -----------------'.format(key))
        for sample in value:
                point, timestamp = sample
                print(timestamp, point)
        print('')

-- bob -----------------
(1508528370.255, 0)
(1508528370.385, 1)
(1508528370.495, 2)
(1508528370.566, 3)
(1508528370.696, 4)
(1508528370.805, 5)
(1508528370.915, 6)
(1508528371.024, 7)
(1508528371.133, 8)
(1508528371.242, 9)

-- sam -----------------
(1508528370.255, [0, 1])
(1508528370.385, [1, 2])
(1508528370.495, [2, 3])
(1508528370.566, [3, 4])
(1508528370.696, [4, 5])
(1508528370.805, [5, 6])
(1508528370.915, [6, 7])
(1508528371.024, [7, 8])
(1508528371.133, [8, 9])
(1508528371.242, [9, 10])

-- b64keys -----------------



Now our two data items `bob` and `tom` which each contain 10 data points with timestamps. The timestamps are in seconds, which hopefully should make sense since we paused a 1/10 of a second between data samples. It won't be perfect, because there is some overhead in moving data around and recording it. C/C++ would be faster, but python is far easier to develop with.

Finally note the last key `b64keys` is empty ... no data. That is an internal data item you don't need to worry about. It basically tracks if there were any binary opencv images that needed to be properly packed for storage or unpacked for retreval.

Now, `bagit` stores data as a [json](https://en.wikipedia.org/wiki/JSON) file which is by default simple text. You can see this by taking a look at the file using the `cat` command. You can even run this command from this notebook as follows:

In [14]:
# let's make sure the file is in the current directory
%ls

 Volume in drive C has no label.
 Volume Serial Number is BC17-351A

 Directory of C:\Users\Kevin.Walchko\github\ece387\website\block_4_mobile_robotics\lsn27

10/20/2017  01:36 PM    <DIR>          .
10/20/2017  01:36 PM    <DIR>          ..
10/20/2017  12:44 PM    <DIR>          .ipynb_checkpoints
10/20/2017  01:36 PM            14,957 lsn27-roomba-overview.ipynb
10/20/2017  01:23 PM               500 my_file.json
10/16/2017  11:55 AM    <DIR>          pics
               2 File(s)         15,457 bytes
               4 Dir(s)  107,967,614,976 bytes free


In [15]:
# now let's read the file
!cat my_file.json

{"bob": [[0, 1508527414.302], [1, 1508527414.402], [2, 1508527414.539], [3, 1508527414.61], [4, 1508527414.752], [5, 1508527414.861], [6, 1508527414.97], [7, 1508527415.079], [8, 1508527415.189], [9, 1508527415.298]], "sam": [[[0, 1], 1508527414.302], [[1, 2], 1508527414.402], [[2, 3], 1508527414.539], [[3, 4], 1508527414.61], [[4, 5], 1508527414.752], [[5, 6], 1508527414.861], [[6, 7], 1508527414.97], [[7, 8], 1508527415.079], [[8, 9], 1508527415.189], [[9, 10], 1508527415.298]], "b64keys": []}


Notice our `dict` keys and data points with timestamps. There is also an optional flag to use compresson on these files to reduce size, but then you can't read them like this.

# Exercises

- Try creating and reading your own bag file

# Questions

1. How do infra red sensors work?
1. How do quadriture encoders work?
1. What are some problems/issues with wheel encoders and dead reckoning?
1. What is a bag file and how does it work?
1. How do you use the roomba software to read its voltage and current?


-----------

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.